In [1]:
from scipy.stats import chi2_contingency
import calendar
import numpy as np
months = list(map(lambda x: x.lower(), calendar.month_name))[1:]
import pandas as pd
from pandas import DataFrame
import statsmodels.api as sm

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pylab as pylab


params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

import os

project_folder = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data/process'
import warnings

warnings.filterwarnings(action='ignore')

from IPython.core.display import display, HTML
# Set the display width to fit the entire notebook width
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import Markdown
from IPython.display import display_html
import scipy.stats as stats
vals_to_removed= ['I prefer to not answer','Don’t know/not sure',None,'nan']
predictors ={'Q10':'is_registered','Q12':'is_insured','Q13':'value_approx','Q14':'is_electric','Q18':'is_recover',  'Q28':'seasons', 'Q29':'purpose', 'age_groups': 'age_groups', 'Q35': 'gender', 'Q36': 'income','Q37':'nm_bikes','Q39':'ethnicity', 'country':'country'}
predictors_names_0 = list(predictors.values())
dependent_vars = {'Q25':'is_replaced','Q30':'mode_alt', 'Q31':'post_act'}
all_vars = predictors.copy()
all_vars.update(dependent_vars)

<span style="color: red;font-size: 50px">RUN THE NEXT CELL FOR THE FIRST TIME ONLY</span>

In [37]:

# Change col names and leave only relevant cols as well as delete unnecessary spaces and parenthesis (except age group)
cols_names = list(all_vars.values())
data_init = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
temp = data_init['age_groups']
d_analysis = data_init.rename(columns=all_vars)[cols_names].astype(str).replace(r"\(.*?\)", "").astype(str).replace(":", "").apply(lambda row: [d.split('(')[0].strip() if '(' in d else d.strip() for d in row])
d_analysis['age_groups'] = temp

# Update age group
groups_name = [
    "Children (<13)",
     "Adolescents (13-17)",
     "Young adults (18-24)",
     "Adults (25-40)",
     "Adults (41-64)",
     "Older adults (>64)"
 ]
def get_age_groups(num):
    if num <13:
        return groups_name[0]
    if num <18:
        return groups_name[1]
    if num < 25:
        return groups_name[2]
    if num < 40:
        return groups_name[3]
    if num < 64:
        return groups_name[4]
    if num > 63:
        return groups_name[5]
    return num
# This method translate the alternative data into how much the alternative is sustainable
def sus_alter(row):
    alt_stat = ['ST','NST']
    if row in  ['Walk','Cycle, personal bicycle','Cycle, rental bicycle','Cycle, public bike share','Micro mobility','Transit']:
        return alt_stat[0]
    elif row in ['Car, as a driver','Car, as a passenger','Taxi / Ride-hailing service','Motorcycle or scooter']:
        return alt_stat[1]
    else:
        return row
d_analysis['age_groups'] = data_init['age'].apply(get_age_groups)
d_analysis['mode_alt'] = d_analysis['mode_alt'].apply(sus_alter)
d_analysis['ethnicity'] = d_analysis['ethnicity'].apply(lambda x:None if x in vals_to_removed else 'European' if x=='European' else 'Other')
d_analysis.to_csv(f'{project_folder}/new_data/new_data_3.csv')
merge_q =d_analysis
merge_q

,is_registered,is_insured,value_approx,is_electric,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,ethnicity,country,is_replaced,mode_alt,post_act
0,Yes,Don’t know/not sure,$1000-$1999,"Yes, with pedal-assist and throttle",No,1.0,Recreation or exercise,Older adults (>64),Woman,"Between $20,000 and $34,999 per year",One,European,CAN,"​Yes, and I replaced it with exactly what was ...",Didn’t make those trips,More often
1,No,No,$2000-3999,"Yes, with pedal-assist and throttle",No,2.0,"Equal mix of recreation, exercise, and transpo...",Older adults (>64),Man,"Between $20,000 and $34,999 per year",Three,European,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
2,No,Yes,$2000-3999,​No,No,1.0,Recreation or exercise,Older adults (>64),Man,"Between $100,000 and $149,999 per year",Two,European,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
3,No,Yes,$500-$999,​No,No,1.0,Recreation or exercise,Older adults (>64),Woman,I prefer to not answer,One,European,CAN,"​Yes, and I replaced it with exactly what was ...",ST,About the same / no change
4,Yes,Yes,$1000-$1999,​No,No,2.0,"Equal mix of recreation, exercise, and transpo...",Older adults (>64),Man,"Between $100,000 and $149,999 per year",Two,European,USA,"​Yes, and I replaced it with exactly what was ...",ST,About the same / no change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,Yes,Yes,$2000-3999,"Yes, with pedal-assist and throttle",No,1.0,"Equal mix of recreation, exercise, and transpo...",NaN,nan,nan,nan,None,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
1817,Don’t know/not sure,No,$250-$499,nan,No,2.0,Transportation,NaN,Prefer not to say,I prefer to not answer,Three,None,USA,"Yes, but the replacement was something cheaper",NST,Less often
1818,No,No,$500-$999,​No,No,1.0,"Equal mix of recreation, exercise, and transpo...",NaN,Woman,"Under $20,000 per year",Two,Other,USA,"Yes, but the replacement was something cheaper",NST,Less often
1819,No,No,$2000-3999,​No,No,nan,nan,NaN,nan,nan,nan,None,CAN,No,nan,nan


In [43]:
data_init = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
# This code create new table which summing up our dependent vars
new_names = ['Is replaced','Mode alternative','Post activity']
all_data=[]
for i,my_var in enumerate(dependent_vars.keys()):
    data = pd.DataFrame(data_init[my_var].value_counts())
    data['per'] = (data/data.sum()*100).round(0)
    data =  data.rename(columns={my_var:'Count'})
    data[new_names[i]] = new_names[i]
    data.index.name = 'Variables'
    data = data.reset_index()
    data  =data.set_index([new_names[i], 'Variables'])
    all_data.append(data)
concatenated_df = pd.concat(all_data)
concatenated_df.to_csv('output_2/data_with_insight/dep_table.csv')

In [37]:
'''dt_df_t= merge_q.drop(columns='Unnamed: 0').fillna(-1)
def to_ordinal_dt(col):
    # This function gets a column and return for each value its ordinal values as it stored in @more_data (for irrelevant data return -1)
    return dt_df[col.name].apply(lambda x:more_data[col.name][0].index(x) if x not in ['I prefer to not answer','Don’t know/not sure','Prefer not to say',-1,'Adolescents (13-17)','Children (<13)','Non-binary / third gender'] else -1 )
dt_df_t_1 = dt_df_t.apply(to_ordinal_dt)
count_per_column = {col: dt_df_t_1[col].value_counts().get(-1, 0) for col in dt_df_t_1.columns}
print(count_per_column)'''


{'value_approx': 4, 'is_electric': 0, 'bicycle_type': 13, 'is_recover': 0, 'seasons': 0, 'purpose': 6, 'age_groups': 0, 'gender': 32, 'income': 164, 'nm_bikes': 11, 'education': 24, 'country': 0, 'is_replaced': 0, 'mode_alt': 22, 'post_act': 0}


<span style="color: Orange;font-size: 50px">Always run the following cell</span>

In [4]:
# read data
merge_q = pd.read_csv(f'{project_folder}/new_data/new_data_3.csv').drop(columns='Unnamed: 0')
# organize and populate @more_data  dictionary
more_data ={item:[list(DataFrame(merge_q[item].unique()).dropna()[0]),False] for  item in all_vars.values()}


# Code relevant for reindex
# Bicycle attributes
q= 'value_approx'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0] = pd.concat([df_t.iloc[0], df_t.iloc[6], df_t.iloc[4], df_t.iloc[8], df_t.iloc[7], df_t.iloc[5], df_t.iloc[1:4].sort_values(by=0, ascending=1)])[0].to_list()

# Demographic

q = 'age_groups'
more_data[q][0] = DataFrame(merge_q[q].unique()).dropna()[::-1][0].to_list()


more_data['seasons'][0] = range(3)
q= 'income'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[6:2:-1], df_t.iloc[8:6:-1], df_t.iloc[9], df_t.iloc[1:3]])[0].to_list()
q = 'nm_bikes'
df_t = DataFrame(merge_q[q].unique()).dropna().sort_values(by=0, ascending=0).reset_index(drop=True)
more_data[q][0]= pd.concat([df_t.iloc[0], df_t.iloc[3], df_t.iloc[1:3], df_t.iloc[5:], df_t.iloc[4]])[0].to_list()
electric = merge_q['is_electric'].unique()
more_data['is_electric'][0]= [electric[1],electric[2], electric[0]]


# for the dependent variable post_act:
more_data['post_act'][0]= ['Less often',
                            'About the same / no change',
                            'More often',
                            'I stopped cycling'
]
# for the dependent variable post_act:
more_data['is_replaced'][0]= ['No',
                              'Yes, but the replacement was something cheaper',
                            '\u200bYes, and I replaced it with exactly what was stolen or something more expensive'
]
more_data['mode_alt'][0]=['ST','NST','Didn’t make those trips']
more_data['is_registered'][0]=['No','Yes','Didn’t make those trips']


{'is_registered': [['No', 'Yes', 'Didn’t make those trips'], False],
 'is_insured': [['Don’t know/not sure', 'No', 'Yes', 'I prefer to not answer'],
  False],
 'value_approx': [['Less than $250',
   '$250-$499',
   '$500-$999',
   '$1000-$1999',
   '$2000-3999',
   '$4000-6999',
   '$7000 or more',
   'Don’t know/not sure',
   'I prefer to not answer'],
  False],
 'is_electric': [['\u200bNo',
   'Yes, with pedal-assist',
   'Yes, with pedal-assist and throttle'],
  False],
 'is_recover': [['No', 'Yes'], False],
 'seasons': [range(0, 3), False],
 'purpose': [['Recreation or exercise',
   'Equal mix of recreation, exercise, and transportation trips',
   'Transportation',
   'Don’t know/not sure'],
  False],
 'age_groups': [['Children (<13)',
   'Adolescents (13-17)',
   'Young adults (18-24)',
   'Adults (25-40)',
   'Adults (41-64)',
   'Older adults (>64)'],
  False],
 'gender': [['Woman', 'Man', 'Non-binary / third gender', 'Prefer not to say'],
  False],
 'income': [['Under $20,000 p

In [5]:
merge_q

,is_registered,is_insured,value_approx,is_electric,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,ethnicity,country,is_replaced,mode_alt,post_act
0,Yes,Don’t know/not sure,$1000-$1999,"Yes, with pedal-assist and throttle",No,1.0,Recreation or exercise,Older adults (>64),Woman,"Between $20,000 and $34,999 per year",One,European,CAN,"​Yes, and I replaced it with exactly what was ...",Didn’t make those trips,More often
1,No,No,$2000-3999,"Yes, with pedal-assist and throttle",No,2.0,"Equal mix of recreation, exercise, and transpo...",Older adults (>64),Man,"Between $20,000 and $34,999 per year",Three,European,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
2,No,Yes,$2000-3999,​No,No,1.0,Recreation or exercise,Older adults (>64),Man,"Between $100,000 and $149,999 per year",Two,European,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
3,No,Yes,$500-$999,​No,No,1.0,Recreation or exercise,Older adults (>64),Woman,I prefer to not answer,One,European,CAN,"​Yes, and I replaced it with exactly what was ...",ST,About the same / no change
4,Yes,Yes,$1000-$1999,​No,No,2.0,"Equal mix of recreation, exercise, and transpo...",Older adults (>64),Man,"Between $100,000 and $149,999 per year",Two,European,USA,"​Yes, and I replaced it with exactly what was ...",ST,About the same / no change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,Yes,Yes,$2000-3999,"Yes, with pedal-assist and throttle",No,1.0,"Equal mix of recreation, exercise, and transpo...",NaN,NaN,NaN,NaN,NaN,USA,"​Yes, and I replaced it with exactly what was ...",NST,About the same / no change
1817,Don’t know/not sure,No,$250-$499,NaN,No,2.0,Transportation,NaN,Prefer not to say,I prefer to not answer,Three,NaN,USA,"Yes, but the replacement was something cheaper",NST,Less often
1818,No,No,$500-$999,​No,No,1.0,"Equal mix of recreation, exercise, and transpo...",NaN,Woman,"Under $20,000 per year",Two,Other,USA,"Yes, but the replacement was something cheaper",NST,Less often
1819,No,No,$2000-3999,​No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAN,No,NaN,NaN


<span style="color: blue;font-size: 25px">Cross tabulation complements </span>

In [12]:
val_1  ='mode_alt'
data_list ={}
for val_2 in merge_q.columns:
    if val_1==val_2:
        continue
    data_list[f'{val_2}'] = round(pd.crosstab(index=merge_q[val_1], columns=merge_q[val_2], normalize='columns'),2).reindex(columns=more_data[val_2][0],index=more_data[val_1][0]) * 100
with pd.ExcelWriter(f"output_2/data_with_insight/dependent/{val_1}_new.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in data_list.items()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

<span style="color: blue;font-size: 50px">Find relationships between independent variables </span>

In [171]:
# dependents_list = ['value_approx','income','nm_bikes','age_groups','education','value_approx','is_electric','is_recover','seasons']

data_to_exp = merge_q[all_vars.values()].fillna(-1)

def to_ordinal(col):
    # This function gets a column and return for each value its ordinal values as it stored in @more_data (for irrelevant data return -1)
    return data_to_exp[col.name].apply(lambda x:more_data[col.name][0].index(x) if x not in ['I prefer to not answer','Don’t know/not sure',-1] else -1)
data_as_ordinal = data_to_exp.apply(to_ordinal)
data_as_ordinal

,is_registered,is_insured,value_approx,is_electric,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,ethnicity,country,is_replaced,mode_alt,post_act
0,1,-1,3,2,0,1,0,5,0,1,1,0,0,2,2,2
1,0,1,4,2,0,2,1,5,1,1,3,0,1,2,1,1
2,0,2,4,0,0,1,0,5,1,5,2,0,1,2,1,1
3,0,2,2,0,0,1,0,5,0,-1,1,0,0,2,0,1
4,1,2,3,0,0,2,1,5,1,5,2,0,1,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816,1,2,4,2,0,1,1,-1,-1,-1,-1,-1,1,2,1,1
1817,-1,1,1,-1,0,2,2,-1,3,-1,3,-1,1,1,1,0
1818,0,1,2,0,0,1,1,-1,0,0,2,1,1,1,1,0
1819,0,1,4,0,0,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1


In [61]:

data_list = []
listy = list(all_vars.values())
# The code test every pair of independent variables to find the direction of the correlation
for items in [(x, y) for i,x in enumerate(listy) for y in listy[i+1:]]:
    var_1, var_2 = items
    df = data_as_ordinal[[var_1, var_2 ]]
    couple_to_test = df[~(df== -1).any(axis=1)] # remove missing or irrelevant data
    correlation, p_value = stats.spearmanr(couple_to_test[var_1],couple_to_test[var_2]) # use spearman to test the data
    data_list.append([var_1,var_2,correlation, p_value])
sta_spearman_table = DataFrame(data_list,columns=['var_1','var_2','correlation', 'p_value']).sort_values('correlation',ascending=False)
sta_spearman_table.to_csv('output_2/data_with_insight/spearman.csv')

<span style="color: blue;font-size: 50px">####</span>

<span style="color: yellow;font-size: 50px">BLR</span>


<span style="color: yellow;font-size: 20px">DT</span>

<span style="color:orange;font-size: 25px">LogisticRegression</span>

In [177]:


class RunLogit:
    def __init__(self):
        self.summary_df = {}
        self.dep=  None
    def run_logit(self,model_data,name):
        print(name)
        X = model_data.drop(columns = my_logit.dep)
        y = model_data[my_logit.dep]
        print(y.value_counts())
        X = sm.add_constant(X)  # Add constant term
        model = sm.Logit(y, X)
        result = model.fit()
        # Print summary with p-values
        print(result.summary())
        df = pd.read_html(result.summary().tables[1].as_html(), header=0, index_col=0)[0]
        my_vars = 'Vars'
        coef = 'Coeff'
        cols_to_save = [my_vars, coef,'[0.025', '0.975]', 'P Value']

        df = df.reset_index()
        df = df.rename(columns={df.columns[0]: my_vars, df.columns[4]: 'P Value'})
        df[coef] = pd.to_numeric(df[df.columns[1]], errors='coerce')

        df = df[cols_to_save]
        df['OR'] = df[coef].apply(lambda x: np.exp(x))

        self.summary_df[name] = df.set_index(my_vars).round(3).drop(index='const')


my_logit = RunLogit()
# Update columns names and
val_to_remove = ['Ethnicity','Is registered','Country','Age groups']
data_as_ordinal.columns = [col.capitalize().replace('_',' ') for col in data_as_ordinal.columns]
data_as_ordinal.rename(columns={'Value approx':'Value approx.','Seasons':'Cycling frequency', 'Nm bikes':'Number of bicycles','Income':'Household Income','Is recover':'Is recovered'},inplace=True)
predictors_names = [name for name in data_as_ordinal.columns if name not in val_to_remove+['Post act','Is replaced','Mode alt']]

# test post_act
my_logit.dep= 'Post act'
data_as_ordinal_model  = data_as_ordinal.copy()[predictors_names+[my_logit.dep]]
data_as_ordinal_model =data_as_ordinal_model[~(data_as_ordinal_model== -1).any(axis=1)]
data_as_ordinal_model[my_logit.dep]= data_as_ordinal_model[my_logit.dep].apply(lambda x:0 if x ==0| x==3 else 1)
my_logit.run_logit(data_as_ordinal_model,my_logit.dep)

# test is_replaced _1

my_logit.dep = 'Is replaced'
data_as_ordinal_model  = data_as_ordinal.copy()[predictors_names+[my_logit.dep]]
data_as_ordinal_model =data_as_ordinal_model[~(data_as_ordinal_model== -1).any(axis=1)]
data_as_ordinal_model[my_logit.dep]= data_as_ordinal_model[my_logit.dep].apply(lambda x: 0 if x==0 else 1)
my_logit.run_logit(data_as_ordinal_model,my_logit.dep)

# test is_replaced 2
data_as_ordinal_model  = data_as_ordinal.copy()[predictors_names+[my_logit.dep]]
data_as_ordinal_model= data_as_ordinal_model[data_as_ordinal_model[my_logit.dep]>0]
data_as_ordinal_model =data_as_ordinal_model[~(data_as_ordinal_model== -1).any(axis=1)]
data_as_ordinal_model[my_logit.dep]= data_as_ordinal_model[my_logit.dep].apply(lambda x: 0 if x==1 else 1)
my_logit.run_logit(data_as_ordinal_model, 'is_replaced_cheap')


# test mode_alt
my_logit.dep = 'Mode alt'
data_as_ordinal_model  = data_as_ordinal.copy()[predictors_names+[my_logit.dep]]
data_as_ordinal_model = data_as_ordinal_model[data_as_ordinal_model[my_logit.dep] < 2]
data_as_ordinal_model =data_as_ordinal_model[~(data_as_ordinal_model== -1).any(axis=1)]
data_as_ordinal_model[my_logit.dep] = data_as_ordinal_model[my_logit.dep].apply(lambda x: 0 if x == 1 else 1)
my_logit.run_logit(data_as_ordinal_model, my_logit.dep)
# Run

with pd.ExcelWriter('output_2/data_with_insight/Logit.xlsx') as writer:
    _ = [item[1].sort_values('P Value').to_excel(writer, sheet_name=item[0]) for item in my_logit.summary_df.items()]

Post act
1    1159
0     189
Name: Post act, dtype: int64
Optimization terminated successfully.
         Current function value: 0.239004
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               Post act   No. Observations:                 1348
Model:                          Logit   Df Residuals:                     1338
Method:                           MLE   Df Model:                            9
Date:                Thu, 14 Sep 2023   Pseudo R-squ.:                  0.4104
Time:                        16:44:54   Log-Likelihood:                -322.18
converged:                       True   LL-Null:                       -546.40
Covariance Type:            nonrobust   LLR p-value:                 6.170e-91
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.5837     

In [153]:
my_logit.summary_df.items()

dict_items([('post_act',                Coef  P Value  [0.025  0.975]  Odd ratio
Vars                                                   
is_insured    0.090    0.708  -0.379   0.558      1.094
value_approx -0.249    0.006  -0.426  -0.072      0.780
is_electric  -0.102    0.602  -0.484   0.281      0.903
is_recover    0.777    0.046   0.014   1.539      2.175
seasons       1.432    0.000   1.063   1.802      4.189
purpose       0.153    0.233  -0.099   0.405      1.166
gender       -0.041    0.811  -0.375   0.294      0.960
income        0.090    0.069  -0.007   0.187      1.094
nm_bikes      1.881    0.000   1.544   2.219      6.562), ('is_replaced',                Coef  P Value  [0.025  0.975]  Odd ratio
Vars                                                   
is_insured    0.412    0.008   0.108   0.717      1.510
value_approx -0.020    0.724  -0.133   0.092      0.980
is_electric  -0.203    0.112  -0.453   0.047      0.817
is_recover   -1.959    0.000  -2.317  -1.602      0.141
seaso

In [155]:
data_as_ordinal_dt

,value_approx,is_electric,is_recover,seasons,purpose,gender,income,nm_bikes,education,is_replaced,mode_alt,post_act
0,3,2,0,1,0,0,1,1,1,2,2,2
1,4,2,0,2,1,1,1,3,2,2,1,1
2,4,0,0,1,0,1,5,2,5,2,1,1
4,3,0,0,2,1,1,5,2,5,2,0,1
5,3,0,0,2,0,1,3,2,5,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1719,2,0,0,1,1,1,3,0,2,0,1,0
1729,1,0,0,1,0,0,1,1,1,2,2,0
1742,4,0,0,1,0,1,7,2,4,2,2,0
1746,2,0,0,1,1,0,7,1,4,0,0,0


In [157]:
pd.crosstab(data_as_ordinal['is_replaced'],data_as_ordinal_model['is_electric'], normalize='columns') * 100

is_electric,0,1,2
is_replaced,,,
0,29.342327,22.580645,35.051546
1,25.379427,14.516129,16.494845
2,45.278246,62.903226,48.453608


In [135]:
data_as_ordinal_dt

,value_approx,is_electric,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,education,is_replaced,mode_alt,post_act
0,3,2,0,1,0,7,0,1,1,1,2,2,2
1,4,2,0,2,1,7,1,1,3,2,2,1,1
2,4,0,0,1,0,7,1,5,2,5,2,1,1
4,3,0,0,2,1,7,1,5,2,5,2,0,1
5,3,0,0,2,0,7,1,3,2,5,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1719,2,0,0,1,1,7,1,3,0,2,0,1,0
1729,1,0,0,1,0,7,0,1,1,1,2,2,0
1742,4,0,0,1,0,7,1,7,2,4,2,2,0
1746,2,0,0,1,1,7,0,7,1,4,0,0,0


In [45]:
def find_sample(x):
    """
    the code search for the leaf under the current node and then return one of its sample
    :param x:
    :return:

    """
    y = clf.tree_.children_right[x]
    if y == -1 & clf.tree_.children_left[x] == -1:
        return features[leaf_id == x][0]
    else:
        return find_sample(y)

In [46]:

def fill_path(row):
    # get all the nodes from for a certain path

    parents_node = []
    node_id  =int(row['node_id'])
    i = node_id
    while i != 0:
        parents_node.append(parent_nodes[i])
        i = parent_nodes[i]
    parents_node.reverse()
    # find one representative sample
    b = find_sample(node_id)
    path_node = []
    j = 0
    for i in parents_node:
        feat_i = clf.tree_.feature[i]
        feat = cols_dt[feat_i]
        threshold = clf.tree_.threshold[i]
        if b[feat_i] <= threshold:
            threshold_sign = "<="
        else:
            threshold_sign = ">"
        path_node.append(f'{j}:{feat} {threshold_sign} {threshold:.4f}')
        j += 1
    path_node.append(
        f'value:{(clf.tree_.value[node_id]/max_data*100).round(0)} samples:{clf.tree_.n_node_samples[node_id]}, impurity:{clf.tree_.impurity[node_id]:.3f}')
    paths_nodes[node_id] = path_node


In [47]:
from sklearn.tree import DecisionTreeClassifier
# Create a Decision Tree classifier for each the three questions which are around post activity
for p in range(-3,0):
    # For some dependent variables number of bike = 0 is not relevant
    if p==-3 or p==-1:
        data_as_ordinal_model = data_as_ordinal_dt[data_as_ordinal_dt['nm_bikes']>0]
    else:
        data_as_ordinal_model = data_as_ordinal_dt
    clf = DecisionTreeClassifier(max_depth=7)
    cols_dt = data_as_ordinal_model .columns
    col_name = cols_dt[p]
    print(col_name)
    print(more_data[col_name])
    features = data_as_ordinal_model [cols_dt[:-3]].to_numpy()
    labels = data_as_ordinal_model[col_name].to_numpy()
    # Build the classifier on the data
    _ =clf.fit(features, labels)

    # feature_importance (the results are not so encouraging)
    new_df = DataFrame(data={'feature_importance':clf.feature_importances_.round(3)*100,'features_name':cols_dt[:-3]}).sort_values('feature_importance',ascending=0)
    print(new_df)
    paths_nodes = {}
    max_data = clf.tree_.value[0]
    leaf_id = clf.apply(features)
    # Create a list so that each node has its parent id
    left_children = clf.tree_.children_left
    right_children = clf.tree_.children_right
    parent_nodes = [None] * clf.tree_.node_count

    for i in range(clf.tree_.node_count):
        if left_children[i] != -1:
            parent_nodes[left_children[i]] = i
        if right_children[i] != -1:
            parent_nodes[right_children[i]] = i

    # Get paths to nodes with at least x samples and square error less than y
    rel_nodes_list = {}
    node_count = clf.tree_.node_count
    for node_id in range(node_count):
        if clf.tree_.n_node_samples[node_id] >= 50 and clf.tree_.impurity[node_id] < 0.4 :
            rel_nodes_list[node_id] =[clf.tree_.value[node_id],clf.tree_.n_node_samples[node_id],clf.tree_.impurity[node_id]]

    # save the data into dateframe
    rel_nodes_list_df = DataFrame.from_dict(rel_nodes_list,orient = 'index' ,columns=['value','n_node_samples','impurity'])
    rel_nodes_list_df = rel_nodes_list_df.reset_index().rename(columns={'index': 'node_id'})
    _ = rel_nodes_list_df.apply(lambda x: fill_path(x),axis= 1)
    print(paths_nodes)

nm_bikes
[['Zero', 'One', 'Two', 'Three', 'Four', 'Five or more', 'I prefer to not answer'], False]
   feature_importance  features_name
2                24.6   value_approx
9                17.3         income
5                16.2        seasons
7                10.6     age_groups
6                 6.7        purpose
3                 5.9    is_electric
0                 5.8  is_registered
8                 4.8         gender
4                 4.2     is_recover
1                 3.9     is_insured
{3: ['0:value_approx <= 2.5000', '1:income <= 2.5000', '2:seasons <= 1.5000', 'value:[[20.  4.  3.  4.  1.]] samples:156, impurity:0.293'], 11: ['0:value_approx <= 2.5000', '1:income <= 2.5000', '2:seasons <= 1.5000', '3:value_approx > 0.5000', 'value:[[18.  3.  3.  2.  0.]] samples:136, impurity:0.251'], 19: ['0:value_approx <= 2.5000', '1:income <= 2.5000', '2:seasons <= 1.5000', '3:value_approx > 0.5000', '4:value_approx > 1.5000', 'value:[[12.  3.  2.  2.  0.]] samples:93, impurity:0.

<span style="color:orange;font-size: 25px">LogisticRegression - Multi</span>


In [ ]:
summary_df= []
cols_dt = data_as_ordinal_dt.columns

for p in range(-3,0):
    # For some dependent variables number of bike = 0 is not relevant
    if p==-2:
        data_as_ordinal_model = data_as_ordinal_dt
    else:
        data_as_ordinal_model = data_as_ordinal_dt[data_as_ordinal_dt['nm_bikes']>0]
    col_name = cols_dt[p]
    print(col_name)
    print(more_data[col_name])
    X = data_as_ordinal_model[cols_dt[:-3]]
    y = data_as_ordinal_model[col_name]
    print(y.value_counts())
    X = sm.add_constant(X)  # Add constant term
    model = sm.MNLogit(y, X)
    result = model.fit()
    # Print summary with p-values
    print(result.summary())
    # Convert the summary table to a pandas DataFrame
    summary_df.append(pd.read_html(result.summary().tables[1].as_html(), header=0, index_col=0)[0])

In [ ]:

# This code arrange the data to be exported to excel (save only data relevant to an interpretation)
my_vars = 'Vars'
coef = 'Coef'
cols_to_save = [my_vars, coef, 'P Value', '[0.025', '0.975]']
data_to_print = {}


def create_new_table(df: DataFrame, table_name: str, start: int, end: (bool, int) = False):
    """
    This method is working on the dataframe (@df) so it -
     1. Reset the index
     2. Update columns name
     3. Get only the relevant row based on the index location in @start and @end
     4. Convert the coefficient column to numeric values
     5. Save only relevant columns' data
     6. Calculate Odd Ratio
     6. Add the new table and its name  into #data_to_print
    """
    print(table_name)
    df = df.reset_index()

    df = df.rename(columns={df.columns[0]: my_vars, df.columns[4]: 'P Value'})
    if end:
        new_table = df.iloc[start:end]
    else:
        new_table = df.iloc[start:]
    new_table[coef] = pd.to_numeric(new_table[new_table.columns[1]], errors='coerce')
    new_table = new_table[cols_to_save]

    new_table['Odd ratio'] = calculate_odd_ratio(new_table)
    new_table = new_table.set_index(my_vars).round(3)
    data_to_print[table_name] = new_table


def calculate_odd_ratio(df):
    return df[coef].apply(lambda x: np.exp(x))


# These three lines  store indices to read only relevant data
len_col_dt = len(cols_dt)
num_of_fctrs = len_col_dt - 3
multi_len_col_dt = 2 * num_of_fctrs + 3
for i, table in enumerate(summary_df):
    table = table.copy()
    old_name = table.index.name
    # This table includes only two or three categories, from every category we create new table
    new_table = create_new_table(table, old_name, 1, num_of_fctrs + 1)
    more_names = table.iloc[len_col_dt - 2].name
    create_new_table(table, more_names, len_col_dt)

# Save the DataFrame to an Excel file
with pd.ExcelWriter('output_2/data_with_insight/MNLogit.xlsx') as writer:
    _ = [item[1].to_excel(writer, sheet_name=item[0]) for item in data_to_print.items()]

In [82]:

# This code arrange the data to be exported to excel (save only data relevant to an interpretation)
my_vars = 'Vars'
coef = 'Coef'
cols_to_save = [my_vars,coef,'P Value','[0.025','0.975]']
data_to_print = {}
def create_new_table(df:DataFrame,table_name:str, start:int,end:(bool,int)=False):
    """
    This method is working on the dataframe (@df) so it -
     1. Reset the index
     2. Update columns name
     3. Get only the relevant row based on the index location in @start and @end
     4. Convert the coefficient column to numeric values
     5. Save only relevant columns' data
     6. Calculate Odd Ratio
     6. Add the new table and its name  into #data_to_print
    """
    print(table_name)
    df = df.reset_index()

    df= df.rename(columns={df.columns[0]: my_vars,df.columns[4]:'P Value'})
    if end:
        new_table = df.iloc[start:end]
    else:
        new_table = df.iloc[start:]
    new_table[coef]=pd.to_numeric(new_table[new_table.columns[1]], errors='coerce')
    new_table= new_table[cols_to_save]

    new_table['Odd ratio'] =  calculate_odd_ratio(new_table)
    new_table= new_table.set_index(my_vars).round(3)
    data_to_print[table_name] = new_table


def calculate_odd_ratio(df):
    return df[coef].apply(lambda x: np.exp(x))

# These three lines  store indices to read only relevant data
len_col_dt = len(cols_dt)
num_of_fctrs = len_col_dt-3
multi_len_col_dt =  2*num_of_fctrs+3
for i,table in enumerate(summary_df):
    table = table.copy()
    old_name = table.index.name
    # This table includes only two or three categories, from every category we create new table
    new_table = create_new_table(table,old_name,1,num_of_fctrs+1)
    more_names = table.iloc[len_col_dt-2].name
    if old_name  =='post_act=1':
        # For three categories
        create_new_table(table,more_names,len_col_dt,multi_len_col_dt)
        create_new_table(table,table.iloc[multi_len_col_dt].name,multi_len_col_dt+2)
    else:
        # For two categories
        create_new_table(table,more_names,len_col_dt)

# Save the DataFrame to an Excel file
with pd.ExcelWriter('output_2/data_with_insight/MNLogit.xlsx') as writer:
   _ =  [item[1].to_excel(writer, sheet_name=item[0]) for item in data_to_print.items()]

is_replaced=1
is_replaced=2
mode_alt=1
mode_alt=2
post_act=1
post_act=2
post_act=3


In [29]:
data_as_ordinal_model

,value_approx,is_electric,bicycle_type,is_recover,seasons,purpose,age_groups,gender,income,nm_bikes,education,country,is_replaced,mode_alt,post_act
0,3,2,0,0,1,0,7,0,1,1,1,0,0,0,3
1,4,2,1,0,2,1,7,1,1,3,2,1,0,1,2
2,4,0,0,0,1,0,7,1,5,2,5,1,0,1,2
4,3,0,3,0,2,1,7,1,5,2,5,1,0,2,2
5,3,0,0,0,2,0,7,1,3,2,5,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,4,0,3,0,1,0,7,0,4,1,2,0,1,1,1
1729,1,0,1,0,1,0,7,0,1,1,1,1,0,0,1
1742,4,0,3,0,1,0,7,1,7,2,4,1,0,0,1
1746,2,0,2,0,1,1,7,0,7,1,4,1,2,2,1


In [31]:
pd.crosstab(data_as_ordinal_model['is_replaced'],data_as_ordinal_model['is_electric'])

is_electric,0,1,2
is_replaced,,,
0,540,38,46
1,307,7,15
2,225,8,27


In [32]:
pd.crosstab(data_as_ordinal_model['is_replaced'],data_as_ordinal_model['purpose'])

purpose,0,1,2
is_replaced,,,
0,278,158,188
1,107,102,120
2,119,64,77


In [33]:
pd.crosstab(data_as_ordinal_model['is_replaced'],data_as_ordinal_model['income'])

income,0,1,2,3,4,5,6,7
is_replaced,,,,,,,,
0,13,26,36,71,95,116,82,185
1,32,34,20,53,44,49,34,63
2,12,14,19,39,42,47,33,54


In [34]:
pd.crosstab(data_as_ordinal_model['is_replaced'],data_as_ordinal_model['value_approx'])

value_approx,0,1,2,3,4,5,6
is_replaced,,,,,,,
0,14,63,169,152,119,63,44
1,15,47,115,77,42,16,17
2,14,40,55,66,55,15,15


In [35]:
pd.crosstab(data_as_ordinal_model['is_replaced'],data_as_ordinal_model['is_recover'])

is_recover,0,1
is_replaced,,
0,553,71
1,305,24
2,143,117


In [76]:
pd.crosstab(data_as_ordinal_model['post_act'],data_as_ordinal_model['purpose'], normalize='columns') * 100

purpose,0,1,2
post_act,,,
0,28.744939,36.129032,29.891304
1,56.275304,51.935484,59.510870
2,8.502024,6.451613,4.891304
3,6.477733,5.483871,5.706522


In [46]:
pd.crosstab(data_as_ordinal_model['post_act'],data_as_ordinal_model['seasons'])

seasons,0,1,2
post_act,,,
0,29,33,8
1,33,283,59
2,20,381,270
3,3,59,22


<span style="color:orange;font-size:25px">###</span>

<span style="color: green;font-size: 50px">More Classification</span>

In [34]:
# Read the social economic columns data
ses_list = ['value_approx','income','nm_bikes','education']
ses = data_as_ordinal[ses_list]
# remove every row with null (-1)
ses_final = ses[~ses.isin([-1]).any(axis=1)]

In [36]:
from sklearn.cluster import AgglomerativeClustering
n=3
my_class = 'class'
cluster = AgglomerativeClustering(n_clusters=n)
ses_final[my_class] = cluster.fit_predict(ses_final.to_numpy())
for x in range(n):
    class_data = ses_final[ses_final[my_class] == x]
    # show the mean for each group and for each season
    class_data.describe()

,value_approx,income,nm_bikes,education,class
count,741.000000,741.000000,741.000000,741.000000,741.0
mean,3.302294,5.987854,2.272605,4.522267,0.0
std,1.442729,1.082346,1.510743,0.575259,0.0
min,0.000000,3.000000,0.000000,2.000000,0.0
25%,2.000000,5.000000,1.000000,4.000000,0.0
50%,3.000000,6.000000,2.000000,5.000000,0.0
75%,4.000000,7.000000,3.000000,5.000000,0.0
max,6.000000,7.000000,5.000000,5.000000,0.0


,value_approx,income,nm_bikes,education,class
count,585.000000,585.000000,585.000000,585.000000,585.0
mean,2.420513,3.647863,1.664957,3.695726,1.0
std,1.360244,1.323820,1.262154,1.340772,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.000000,3.000000,1.000000,3.000000,1.0
50%,2.000000,4.000000,1.000000,4.000000,1.0
75%,3.000000,4.000000,2.000000,5.000000,1.0
max,6.000000,7.000000,5.000000,5.000000,1.0


,value_approx,income,nm_bikes,education,class
count,196.000000,196.000000,196.000000,196.000000,196.0
mean,2.025510,0.795918,0.979592,3.443878,2.0
std,1.029992,0.729723,0.671464,1.359454,0.0
min,0.000000,0.000000,0.000000,0.000000,2.0
25%,1.000000,0.000000,1.000000,3.000000,2.0
50%,2.000000,1.000000,1.000000,4.000000,2.0
75%,3.000000,1.000000,1.000000,4.000000,2.0
max,5.000000,2.000000,3.000000,5.000000,2.0


In [41]:
my_info = class_data.describe()
my_mean = my_info.loc['mean']
my_std = my_info.loc['std']
my_min = my_mean- my_std
my_max= my_mean+ my_std


value_approx    3.055503
income          1.525642
nm_bikes        1.651055
education       4.803332
class           2.000000
dtype: float64

In [38]:
more_data['education']

[['Some high school or less',
  'Graduated high school',
  'Some university',
  'Associate’s/vocational/technical\u202fdegree',
  'Bachelor’s degree',
  'Graduate degree',
  'I prefer to not answer'],
 False]

<span style="color: green;font-size: 50px">####</span>

In [8]:

class MyData:
    """
   The class organizes and explores the data, allowing for the creation of cross-tabulations.
    """
    def __init__(self, var_0, data, com_data):
        """

        :param var_0: The main variable that should not be replaced frequently throughout the program.
        :param data: Data frame
        :param com_data: provides more information when necessary to create more adaptable analysis
        """
        self.cols_name = [var_0,'']
        self.merge_q= data
        self.more_data = com_data
        self.reindex_rows = self.more_data[var_0][0]

    def explore_data(self,cross_tab= True):
        r"""
        Clean the data and print cross_tab if it is required
        :param cross_tab:
        :return:
        """
        # Based on these variables, the analysis can be employed.
        cross_cols_nm =self.cols_name
        cls_to_use = self.more_data[cross_cols_nm[1]][1]
        reindex_temp = self.more_data[cross_cols_nm[1]][0]
        only_rel_f= self.merge_q[cross_cols_nm].dropna()
        if cross_tab:
            if cls_to_use:
                only_rel_f = only_rel_f[only_rel_f[cross_cols_nm[1]].isin(cls_to_use)]
            # For ordinal categories, reindexing the columns is essential to align them in the required order.
            return only_rel_f[cross_cols_nm[1]].value_counts().reindex(reindex_temp), (pd.crosstab(only_rel_f[cross_cols_nm[0]], only_rel_f[cross_cols_nm[1]], normalize='columns') * 100).astype(int).reindex(columns=reindex_temp,index= self.reindex_rows)
        return only_rel_f

    def change_properties(self,name):
        """
        change the dependent variable and update desired reindex list
        :param name:
        :return:
        """
        self.cols_name[0] = name
        self.reindex_rows = self.more_data[name][0]
        sumy =my_data.merge_q[name].value_counts().sum()
        print((my_data.merge_q[name].value_counts()/sumy*100).apply('{:.0f}%'.format))




In [13]:

dependent_names = list(dependent_vars.values())+['value_approx','income','nm_bikes','age_groups','education','seasons','is_electric','purpose']
predictors_names = list(predictors.values())
my_data = MyData(dependent_names[0],merge_q,more_data)
for dep in dependent_names:
    print(f'\n{dep}\n')
    writer = pd.ExcelWriter(f'output_2/data_with_insight/{dep}.xlsx')
    my_data.change_properties(dep)
    for pre in predictors_names+dependent_names:
        # relevant when we analysis the relationship between dependent_vars
        if pre==dep:
            continue
        my_data.cols_name[1] = pre
        value_counts,cross_tab = my_data.explore_data()
        value_counts.to_excel(writer, sheet_name=pre)
        cross_tab.to_excel(writer, sheet_name=pre, startrow=value_counts.shape[0] + 2)
    writer.save()



is_replaced

​Yes, and I replaced it with exactly what was stolen or something more expensive    46%
No                                                                                  31%
Yes, but the replacement was something cheaper                                      24%
Name: is_replaced, dtype: object

mode_alt

not active                 50%
active                     36%
Didn’t make those trips    13%
Don’t know/not sure         2%
Name: mode_alt, dtype: object

post_act

About the same / no change    49%
Less often                    30%
I stopped cycling             15%
More often                     6%
Name: post_act, dtype: object

value_approx

$500-$999                 29%
$1000-$1999               24%
$2000-3999                16%
$250-$499                 13%
$4000-6999                 8%
$7000 or more              5%
Less than $250             4%
Don’t know/not sure        0%
I prefer to not answer     0%
Name: value_approx, dtype: object

income

$200,000 or more pe

<span style="color: purple;font-size: 50px">Phase 4: Apply Chi-Square Test</span>


In [3]:
#region
cols_to_delete = ['I prefer to not answer', 'Don’t know/not sure']
pre_test_data = pd.read_csv(f'{project_folder}/new_data/new_data_3.csv').fillna(-1)[list(all_vars.values())].applymap(lambda x: -1 if x in cols_to_delete + [-1] else x)
data_path = 'output_2/data_with_insight'
class MyTests:
    def __init__(self, col_1:str, col_2:str, rel_data:tuple):
        print(f'\n{col_1}::{col_2}')
        self.col_1 = col_1
        self.col_2 = col_2
        self.fields_to_test= rel_data[1]
        self.rel_data = rel_data

        if 'apply_chi' in rel_data[0]:
            print('apply_chi')
            self.__apply_chi()
        else:
            print('apply_spearmanr')
            self.__apply_spearmanr( rel_data[2])

    def __apply_chi(self):
        """
        apply chi_square test based on the cols and data
        :param new_names: more relevant data to employ when run the test
        :param col_1:
        :param col_2:
        :return:
        """
        # b.	Clean the data
        test_data = pre_test_data[[self.col_1, self.col_2]]
        # In case more data should be remove prior to the analysis
        if isinstance(self.rel_data[-1],list):
            test_data = test_data[~test_data.isin([-1]+self.rel_data[-1]).any(axis=1)]
        else:
            test_data = test_data[~test_data.isin([-1]).any(axis=1)]

        # c.	Create group
        for item in self.fields_to_test.items():
            internal_dict  = item[1]
            var_temp = item[0]
            test_data[var_temp] =  test_data[var_temp].apply(lambda x: internal_dict[x] if x in internal_dict else x)

        # d.	Cross tub
        contingency_table = pd.crosstab(test_data[self.col_1], test_data[self.col_2], normalize='columns') * 100

        # e.	test
        chi2, p, _, _ = stats.chi2_contingency(contingency_table)

        # Print the chi-square test statistic and p-value
        print(contingency_table)
        print('Chi-square test statistic:', chi2)
        print('p-value:', p)

    def __apply_spearmanr(self, more_cols_to_delete):

        # The data to read (the Excel name- @col_1 and sheet name  - @col_2)
        df = pd.read_excel(f'{data_path}/{self.col_1}.xlsx', sheet_name=self.col_2)
        # Obten only the contingency_table from the file, update the cols names and remove irrelevant data
        new_df = df.iloc[df.loc[df[df.columns[0]].isnull()].index.item()+1:]
        new_df.columns= new_df.iloc[0]
        cols_to_delete_temp = [col for col in cols_to_delete if col in new_df.columns] + more_cols_to_delete
        new_df = new_df.reset_index(drop=True).drop(0).drop(columns= cols_to_delete_temp).set_index(var_1)
        # extract the required data for test
        data_1= np.array(range(len(new_df.columns)))
        # the data to test which can be based on one or more fields (if it is list is more than one)
        if isinstance(self.fields_to_test,list):
            data_2 = np.array(new_df.loc[self.fields_to_test].sum())
        else:
            data_2= np.array(new_df.loc[self.fields_to_test])
        # Perform the Cochran-Armitage test of trend
        result = stats.spearmanr(data_1, data_2)

        # Extract the test statistic and p-value
        test_statistic = result.correlation
        p_value = result.pvalue

        # Print the test statistic and p-value
        print("Test Statistic:", test_statistic)
        print("p-value:", p_value)
#endregion

In [3]:
predictors ={'Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q18':'is_recover',  'Q28':'seasons', 'Q29':'purpose', 'age_groups': 'age_groups', 'Q35': 'gender', 'Q36': 'income','Q37':'nm_bikes', 'Q38': 'education','country':'country'}
dependent_vars = {'Q25':'is_replaced','Q30':'mode_alt', 'Q31':'post_act'}

In [ ]:
var_1 = 'mode_alt'
vars_2 = ['value_approx','nm_bikes','age_groups','income']
for var_2 in vars_2:
    for temp_val in pre_test_data[pre_test_data[var_1 ] != -1][var_1].drop_duplicates().values:
        print(f'\n{temp_val}')
        if var_2=='age_groups':
            test_fields=('spearmanr', temp_val,['Children (<13)'])
        else:
            test_fields=('spearmanr', temp_val,[])
        res = MyTests(var_1, var_2, test_fields)

In [8]:
pre_test_data[pre_test_data['is_replaced'] != -1]['is_replaced'].drop_duplicates().values

array(['\u200bYes, and I replaced it with exactly what was stolen or something more expensive',
       'Yes, but the replacement was something cheaper', 'No'],
      dtype=object)

In [5]:
#region
# dependent 2
var_1 = 'mode_alt'

var_2 = 'seasons'
val_1 = 'not active'
test_fields= ('apply_chi', {var_1:{'Didn’t make those trips':val_1}})
MyTests(var_1, var_2, test_fields)

var_2= 'purpose'
test_fields= ('apply_chi', {var_1:{'Didn’t make those trips':val_1}})
MyTests(var_1, var_2, test_fields)

val_1 = 'make this trip'
test_fields= ('apply_chi', {var_1:{'active':val_1, 'not active':val_1}})
MyTests(var_1, var_2, test_fields)

var_2 = 'is_electric'
val_1 = 'active'
test_fields=('apply_chi', {var_1:{'Didn’t make those trips':val_1}})
MyTests(var_1, var_2, test_fields)

var_2 = 'value_approx'
val_1 = 'active'
test_fields=('spearmanr', val_1,['$7000 or more'])
MyTests(var_1, var_2, test_fields)

var_2 = 'is_recover'
val_1 = 'not active'
test_fields=('apply_chi', {var_1:{'Didn’t make those trips':val_1}})
MyTests(var_1, var_2, test_fields)

var_2 = 'nm_bikes'
val_1 = 'active'
test_fields=('spearmanr', val_1,[])
MyTests(var_1, var_2, test_fields)

var_2 = 'age_groups'
val_1 = 'not active'
val_2 = 'not young'
test_fields=('apply_chi', {var_1:{'Didn’t make those trips':val_1},var_2:{age:val_2 for age in pre_test_data[var_2]if age not  in ['Young adults (18-24)']}},['Children (<13)','Adolescents (13-17)'])
MyTests(var_1, var_2, test_fields)

var_2 = 'income'
val_1 = 'not active'
val_2 = 'not low income'
test_fields=('apply_chi', {var_1:{'Didn’t make those trips':val_1},var_2:{b:val_2 for b in pre_test_data[var_2]if b not  in ['Under $20,000 per year']}})
MyTests(var_1, var_2, test_fields)
#endregion


mode_alt::seasons
apply_chi
seasons      0.0        1.0        2.0
mode_alt                              
active      25.0  29.949749  55.251142
not active  75.0  70.050251  44.748858
Chi-square test statistic: 22.659060879893193
p-value: 1.2012888535235022e-05



mode_alt::purpose
apply_chi
purpose     Equal mix of recreation, exercise, and transportation trips  \
mode_alt                                                                  
active                                              39.159292             
not active                                          60.840708             

purpose     Recreation or exercise  Transportation  
mode_alt                                            
active                   31.518625       41.111111  
not active               68.481375       58.888889  
Chi-square test statistic: 2.1987514330944222
p-value: 0.33307895448593955



mode_alt::purpose
apply_chi
purpose                  Equal mix of recreation, exercise, and transportation trips  \
mode_alt                                                                               
Didn’t make those trips                                            4.20354             
make this trip                                                    95.79646             

purpose                  Recreation or exercise  Transportation  
mode_alt                                                         
Didn’t make those trips               26.790831        2.407407  
make this trip                        73.209169       97.592593  
Chi-square test statistic: 37.326712115701845
p-value: 7.845261971756796e-09



mode_alt::is_electric
apply_chi
is_electric  Yes, with pedal-assist  Yes, with pedal-assist and throttle  \
mode_alt                                                                   
active                    39.772727                            33.333333   
not active                60.227273                            66.666667   

is_electric        ​No  
mode_alt                
active       51.407001  
not active   48.592999  
Chi-square test statistic: 6.912631294337069
p-value: 0.03154577393423454



mode_alt::value_approx
apply_spearmanr
Test Statistic: -0.942857142857143
p-value: 0.0048046647230320545



mode_alt::is_recover
apply_chi
is_recover         No        Yes
mode_alt                        
active      34.610028  46.564885
not active  65.389972  53.435115
Chi-square test statistic: 2.4883616460247038
p-value: 0.1146910604066402



mode_alt::nm_bikes
apply_spearmanr
Test Statistic: 0.942857142857143
p-value: 0.0048046647230320545



mode_alt::age_groups
apply_chi
age_groups  Young adults (18-24)  not young
mode_alt                                   
active                      50.0  35.669192
not active                  50.0  64.330808
Chi-square test statistic: 3.628732917278225
p-value: 0.05679003728169556



mode_alt::income
apply_chi
income      Under $20,000 per year  not low income
mode_alt                                          
active                   51.219512       35.638673
not active               48.780488       64.361327
Chi-square test statistic: 4.32674329570235
p-value: 0.03751796448189743


In [11]:
var_1 = 'mode_alt'
vars_2 = ['value_approx','nm_bikes','age_groups','income']
for var_2 in vars_2:
    for temp_val in pre_test_data[pre_test_data[var_1 ] != -1][var_1].drop_duplicates().values:
        print(f'\n{temp_val}')
        if var_2=='age_groups':
            test_fields=('spearmanr', temp_val,['Children (<13)'])
        else:
            test_fields=('spearmanr', temp_val,[])
        res = MyTests(var_1, var_2, test_fields)


Didn’t make those trips

mode_alt::value_approx
apply_spearmanr
Test Statistic: 0.4504687313477795
p-value: 0.3104293018513229

not active

mode_alt::value_approx
apply_spearmanr
Test Statistic: -0.18018749253911182
p-value: 0.6990457740939315

active

mode_alt::value_approx
apply_spearmanr
Test Statistic: -0.6071428571428572
p-value: 0.1482311614811614

Didn’t make those trips

mode_alt::nm_bikes
apply_spearmanr
Test Statistic: 0.16903085094570333
p-value: 0.7488684500235265

not active

mode_alt::nm_bikes
apply_spearmanr
Test Statistic: -0.7944613465542746
p-value: 0.05902759992532669

active

mode_alt::nm_bikes
apply_spearmanr
Test Statistic: 0.942857142857143
p-value: 0.0048046647230320545

Didn’t make those trips

mode_alt::age_groups
apply_spearmanr
Test Statistic: 0.4642857142857144
p-value: 0.2939341076002517

not active

mode_alt::age_groups
apply_spearmanr
Test Statistic: -0.36037498507822363
p-value: 0.42714880856090626

active

mode_alt::age_groups
apply_spearmanr
Test Sta

In [7]:
#region
# dependent 3
var_1 ='post_act'

# a.	Define var
var_2 = 'is_recover'
val_1 = 'positive'
val_2 = 'negative'

test_fields= ('apply_chi', {var_1:{'I stopped cycling':val_2, 'Less often':val_2,'About the same / no change':val_1,'More often':val_1}})
res= MyTests(var_1, var_2, test_fields)
var_2 = 'seasons'
test_fields=('apply_chi', {var_1:{'I stopped cycling':val_2, 'Less often':val_2,'About the same / no change':val_1,'More often':val_1}})
res = MyTests(var_1, var_2, test_fields)
test_fields=('apply_chi', {var_1:{'I stopped cycling':val_2, 'Less often':val_1,'About the same / no change':val_1,'More often':val_1},var_2:{2:1}})
res = MyTests(var_1, var_2, test_fields)

var_2 = 'age_groups'
val_1 = 'still cycling'

test_fields=('apply_chi', {var_1:{ 'Less often':val_1,'About the same / no change':val_1,'More often':val_1},var_2:{age:val_2 for age in pre_test_data[var_2]if age not  in ['Young adults (18-24)']}},['Children (<13)','Adolescents (13-17)'])
res = MyTests(var_1, var_2, test_fields)
var_2 = 'income'
test_fields=('spearmanr', 'I stopped cycling',[])
res = MyTests(var_1, var_2, test_fields)
var_2 = 'value_approx'

val_1 = 'no more often'
val_2 = 'not cheap bike'
test_fields=('apply_chi', {var_1:{ 'Less often':val_1,'About the same / no change':val_1,'I stopped cycling':val_1},var_2:{temp:val_2 for temp in pre_test_data[var_2]if temp not  in ['Less than $250']}})
res = MyTests(var_1, var_2, test_fields)
var_2 = 'seasons'
val_1 = 'negative'
test_fields=('apply_chi', {var_1:{ 'Less often':val_1,'I stopped cycling':val_1}},['More often'])
res = MyTests(var_1, var_2, test_fields)
var_2 = 'value_approx'
test_fields=('spearmanr', 'About the same / no change',[])
res = MyTests(var_1, var_2, test_fields)

var_2 = 'nm_bikes'
test_fields=('spearmanr', 'About the same / no change',['Zero'])
res = MyTests(var_1, var_2, test_fields)

var_2 = 'education'
test_fields=('spearmanr', 'About the same / no change',[])
res = MyTests(var_1, var_2, test_fields)
#endregion


post_act::is_recover
apply_chi
is_recover       No        Yes
post_act                      
negative    47.8738  27.675277
positive    52.1262  72.324723
Chi-square test statistic: 7.840402304773549
p-value: 0.005109123495550274

post_act::seasons
apply_chi
seasons         0.0        1.0        2.0
post_act                                 
negative  82.439024  45.986125  22.121896
positive  17.560976  54.013875  77.878104
Chi-square test statistic: 73.82059854544553
p-value: 9.333847489738014e-17

post_act::seasons
apply_chi
seasons         0.0        1.0
post_act                      
negative  58.536585   8.539945
positive  41.463415  91.460055
Chi-square test statistic: 53.8505747481379
p-value: 2.1633180147558866e-13

post_act::age_groups
apply_chi
age_groups         Young adults (18-24)   negative
post_act                                          
I stopped cycling             32.258065  14.374226
still cycling                 67.741935  85.625774
Chi-square test statistic: 7.97

In [5]:
var_1 = 'post_act'
vars_2 = ['age_groups','income','nm_bikes','value_approx','education']
for var_2 in vars_2:
    for temp_val in pre_test_data[pre_test_data[var_1 ] != -1][var_1].drop_duplicates().values:
        print(f'\n{temp_val}')
        if var_2=='age_groups':
            test_fields=('spearmanr', temp_val,['Children (<13)'])
        elif var_2=='nm_bikes':
            test_fields=('spearmanr', temp_val,['Zero'])
        else:
            test_fields=('spearmanr', temp_val,[])
        res = MyTests(var_1, var_2, test_fields)


More often

post_act::age_groups
apply_spearmanr
Test Statistic: -0.10910894511799621
p-value: 0.8158714843641036

About the same / no change

post_act::age_groups
apply_spearmanr
Test Statistic: 0.7500000000000002
p-value: 0.05218140045705776

Less often

post_act::age_groups
apply_spearmanr
Test Statistic: -0.3243374865704013
p-value: 0.47788547973993134

I stopped cycling

post_act::age_groups
apply_spearmanr
Test Statistic: -0.14285714285714288
p-value: 0.7599453002180929

More often

post_act::income
apply_spearmanr
Test Statistic: -0.6747477697966319
p-value: 0.06640156509474052

About the same / no change

post_act::income
apply_spearmanr
Test Statistic: 0.9700772721497397
p-value: 6.548558831120658e-05

Less often

post_act::income
apply_spearmanr
Test Statistic: 0.06024533658898499
p-value: 0.8873130218901896

I stopped cycling

post_act::income
apply_spearmanr
Test Statistic: -0.9221722216732094
p-value: 0.0011108173578061712

More often

post_act::nm_bikes
apply_spearmanr
T

In [10]:
pre_test_data['age_groups'].unique()


array(['Older adults (>64)', 'Adults (55-64)', 'Adults (45-54)',
       'Adults (35-44)', 'Adults (25-34)', 'Young adults (18-24)',
       'Adolescents (13-17)', 'Children (<13)'], dtype=object)

In [9]:
#region
# dependent 1
var_1 ='is_replaced'

In [ ]:

# a.	Define var
var_2 = 'is_recover'
val = 'yes'
test_fields= ('apply_chi', {var_1:{'​Yes, and I replaced it with exactly what was stolen or something more expensive':val, 'Yes, but the replacement was something cheaper':val}})
res= MyTests(var_1, var_2, test_fields)

var_2 = 'seasons'
test_fields=('apply_chi', {var_1:{'​Yes, and I replaced it with exactly what was stolen or something more expensive':val, 'Yes, but the replacement was something cheaper':val}, var_2:{2:1}})
res = MyTests(var_1, var_2, test_fields)


var_2 = 'value_approx'
val_1= 'low'
val_2 = 'high'
test_fields=('apply_chi', {var_1:{'​Yes, and I replaced it with exactly what was stolen or something more expensive':val, 'Yes, but the replacement was something cheaper':'No'}, var_2:{'Less than $250':val_1,'$250-$499':val_1,'$500-$999':val_1,'$1000-$1999':val_1,'$2000-3999':val_1,'$4000-6999':val_2,'$7000 or more':val_2}})
res = MyTests(var_1, var_2, test_fields)

In [6]:
var_1 = 'is_replaced'
vars_2 = ['income','nm_bikes','education','value_approx']
for var_2 in vars_2:
    for temp_val in pre_test_data[pre_test_data[var_1 ] != -1][var_1].drop_duplicates().values:
        print(f'\n{temp_val}')
        if var_2=='nm_bikes':
            test_fields=('spearmanr', temp_val,['Zero'])
        else:
            test_fields=('spearmanr', temp_val,[])
        res = MyTests(var_1, var_2, test_fields)

#endregion


​Yes, and I replaced it with exactly what was stolen or something more expensive

is_replaced::income
apply_spearmanr
Test Statistic: 0.9700772721497397
p-value: 6.548558831120658e-05

Yes, but the replacement was something cheaper

is_replaced::income
apply_spearmanr
Test Statistic: -0.8982196964349441
p-value: 0.002438796796382353

No

is_replaced::income
apply_spearmanr
Test Statistic: -0.9157291161525718
p-value: 0.0014031743248924412

​Yes, and I replaced it with exactly what was stolen or something more expensive

is_replaced::nm_bikes
apply_spearmanr
Test Statistic: 0.8999999999999998
p-value: 0.03738607346849874

Yes, but the replacement was something cheaper

is_replaced::nm_bikes
apply_spearmanr
Test Statistic: -0.9746794344808964
p-value: 0.004818230468198537

No

is_replaced::nm_bikes
apply_spearmanr
Test Statistic: -0.44721359549995787
p-value: 0.45018485575210093

​Yes, and I replaced it with exactly what was stolen or something more expensive

is_replaced::education
app

<span style="color: blue;font-size: 50px">Make table of the main variables with the main options</span>


In [8]:
predictors ={'Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q18':'is_recover',  'Q28':'seasons', 'Q29':'purpose', 'age_groups': 'age_groups', 'Q35': 'gender', 'Q36': 'income','Q37':'nm_bikes', 'Q38': 'education','country':'country'}
dependent_vars = {'Q25':'is_replaced','Q30':'mode_alt', 'Q31':'post_act'}

In [9]:
cols_1 = list(dependent_vars.values())
cols_2 = ['value_approx','is_recover','seasons', 'income','nm_bikes','education']


In [10]:
col_1= cols_1[0]
col_2 = cols_2[0]
df = pd.read_excel(f'{data_path}/{col_1}.xlsx', sheet_name=col_2)
df.iloc[df.loc[df[df.columns[0]].isnull()].index.item()+1:]

,Unnamed: 0,value_approx,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
10,is_replaced,Less than $250,$250-$499,$500-$999,$1000-$1999,$2000-3999,$4000-6999,$7000 or more,Don’t know/not sure,I prefer to not answer
11,"​Yes, and I replaced it with exactly what was ...",41,36,41,45,51,58,59,40,50
12,"Yes, but the replacement was something cheaper",25,27,26,23,18,16,18,20,50
13,No,32,36,31,30,30,24,21,40,0


In [43]:

df = pd.read_excel(f'{data_path}/{col_1}.xlsx', sheet_name=col_2)
# Obten only the contingency_table from the file, update the cols names and remove irrelevant data
new_df = df.iloc[df.loc[df[df.columns[0]].isnull()].index.item()+1:]

new_df.columns= new_df.iloc[0]
cols_to_delete_temp = [col for col in cols_to_delete if col in new_df.columns]
new_df = new_df.reset_index(drop=True).drop(0).drop(columns= cols_to_delete_temp).set_index(col_1).drop(index= 'No')
new_df_1 = new_df.set_index(col_1*len(new_df))
new_df_1


KeyError: "None of ['is_replacedis_replaced'] are in the columns"

In [38]:
cols_to_delete

['I prefer to not answer', 'Don’t know/not sure']

In [ ]:

new_df = new_df.reset_index(drop=True).drop(0).drop(columns= cols_to_delete_temp).set_index(col_1)
# extract the required data for test

new_df.drop(index= 'No')

In [12]:
cols_to_delete_temp

['I prefer to not answer', 'Don’t know/not sure']

<span style="color: red;font-size: 50px">Test Zone</span>

In [244]:
# dependent 3
var_1 ='post_act'

# a.	Define var
var_2 = 'is_recover'
val_1 = 'positive'
val_2 = 'negative'
col_1 = 'post_act'
col_2='seasons'
fields_to_test= {var_1:{'I stopped cycling':val_2, 'Less often':val_2,'About the same / no change':val_1,'More often':val_1}}
test_data = pre_test_data[[col_1, col_2]]
test_data = test_data[~test_data.isin([-1]).any(axis=1)]
test_data

,post_act,seasons
0,More often,1.0
1,About the same / no change,2.0
2,About the same / no change,1.0
3,About the same / no change,1.0
4,About the same / no change,2.0
...,...,...
1813,About the same / no change,0.0
1816,About the same / no change,1.0
1817,Less often,2.0
1818,Less often,1.0


In [245]:

# c.	Create group
for item in fields_to_test.items():
    internal_dict  = item[1]
    var_temp = item[0]
    test_data[var_temp] =  test_data[var_temp].apply(lambda x: internal_dict[x] if x in internal_dict else x)
test_data

,post_act,seasons
0,positive,1.0
1,positive,2.0
2,positive,1.0
3,positive,1.0
4,positive,2.0
...,...,...
1813,positive,0.0
1816,positive,1.0
1817,negative,2.0
1818,negative,1.0


In [243]:
internal_dict

{'I stopped cycling': 'high',
 'Less often': 'high',
 'About the same / no change': 'low',
 'More often': 'low'}

In [246]:

# d.	Cross tub
contingency_table = pd.crosstab(test_data[col_1], test_data[col_2], normalize='columns') * 100
contingency_table

seasons,0.0,1.0,2.0
post_act,,,
negative,82.439024,45.986125,22.121896
positive,17.560976,54.013875,77.878104


In [247]:

# e.	test
chi2, p, _, _ = stats.chi2_contingency(contingency_table)

# Print the chi-square test statistic and p-value
print(contingency_table)
print('Chi-square test statistic:', chi2)
print('p-value:', p)

seasons         0.0        1.0        2.0
post_act                                 
negative  82.439024  45.986125  22.121896
positive  17.560976  54.013875  77.878104
Chi-square test statistic: 73.82059854544553
p-value: 9.333847489738014e-17
